# Exercise 2: AES-ECB Weaknesses and Password Entropy

## Objectives
1. **Password Entropy Calculation** - Determine required length for secure passwords
2. **AES-ECB Mode Attack** - Exploit Electronic Codebook mode weaknesses
3. **Pattern Recognition** - Identify encryption vulnerabilities through block analysis

## Part 1: Password Entropy

### What is Entropy?
Measure of unpredictability/randomness in bits. For passwords:

$$H = \log_2(N)$$

where $N$ = number of possible passwords

### Character Set Entropy
For password of length $L$ from alphabet of size $A$:

$$H = L \times \log_2(A)$$

## Exercise 1: AES-256 Key Entropy

### Problem
How many lowercase letters [a-z] needed for 256-bit entropy?

### Solution
```python
entropy_per_char = log2(26) ≈ 4.70 bits
required_length = 256 / 4.70 ≈ 54.5
```

**Answer**: Need at least **55 characters** from [a-z]

### Comparison
| Character Set | Size | Entropy/char | Length for 256 bits |
|---------------|------|--------------|---------------------|
| [a-z] | 26 | 4.70 | 55 |
| [a-zA-Z] | 52 | 5.70 | 45 |
| [a-zA-Z0-9] | 62 | 5.95 | 44 |
| [a-zA-Z0-9+symbols] | 94 | 6.55 | 40 |

## Part 2: AES-ECB Mode

### What is ECB (Electronic Codebook)?
- Simplest block cipher mode
- Each block encrypted independently with same key
- **No chaining** between blocks

```
Plaintext:  [Block1] [Block2] [Block3]
            ↓        ↓        ↓
Encrypt:    AES(K)   AES(K)   AES(K)
            ↓        ↓        ↓
Ciphertext: [Enc1]   [Enc2]   [Enc3]
```

### Critical Weakness
**Identical plaintext blocks → Identical ciphertext blocks**

This reveals patterns in original data!

## Attack 1: Brute Force with Weak Keys

### Scenario
Key generated by repeating single character 16 times:
```python
key = 'a' * 16  # "aaaaaaaaaaaaaaaa"
key = 'b' * 16  # "bbbbbbbbbbbbbbbb"
```

### Key Space
- Only 26 possible keys (for [a-z])
- Can test all in < 1 second

### Validation Method
BMP files start with signature `'BM'` (0x42 0x4D):
```python
if decrypted_data.startswith(b'BM'):
    # Valid BMP file!
```

## Attack 2: ECB Pattern Analysis

### Block Repetition Analysis
Count how many times each 16-byte block appears:

```python
blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
uniqueness = len(set(blocks)) / len(blocks)
```

### Indicators of ECB
- **Low uniqueness** (< 90%) → likely ECB mode
- **High repetition** of specific blocks → uniform areas in image
- **Pattern preservation** → can reconstruct rough image outline

### Example Results
For encrypted BMP:
- Total blocks: 7,737
- Unique blocks: 2,995 (38.7%)
- One block appears **3,761 times** (48.6%)!

## Attack 3: Template Attack

### Known File Structure
BMP file format (first 54 bytes):
```
Offset | Size | Description
-------|------|-------------
0x00   | 2    | Signature 'BM'
0x02   | 4    | File size
0x06   | 4    | Reserved (0)
0x0A   | 4    | Data offset
...
```

### Attack Strategy
1. Try each possible key
2. Decrypt first block (16 bytes)
3. Check if starts with 'BM'
4. Verify file structure fields

**Advantage**: Works without seeing original file!

## Attack 4: Differential Cryptanalysis

### Concept
Compare multiple ciphertexts to find commonalities

### Method
1. Collect multiple files encrypted with same key
2. Look for identical blocks across files
3. Deduce information about plaintexts

### Effectiveness
- **Single file**: Limited information
- **Multiple files**: Can identify common content
- **Same key**: Reveals which parts of files are identical

## Practical Demonstration

### The Famous ECB Penguin
Classic example showing ECB weakness:

Original image → Encrypted with ECB → **Penguin outline still visible!**

Why? Because:
- White background blocks all encrypt to same ciphertext
- Black penguin blocks encrypt differently
- Pattern is preserved despite encryption

### CBC vs ECB
```
ECB: Patterns visible in ciphertext
CBC: Looks like random noise (correct!)
```

## Security Recommendations

### Key Generation
- **DON'T**: Repeat single character
- **DO**: Use cryptographically secure random generator
```python
import secrets
key = secrets.token_bytes(16)  # 128-bit random key
```

### Block Cipher Modes
- **NEVER use ECB** for anything except key wrapping
- **Recommended**: GCM (authenticated encryption)
- **Alternatives**: CBC, CTR with proper IV

### Additional Protections
1. **Authentication**: Use HMAC or GCM built-in auth
2. **IV/Nonce**: Generate random, never reuse
3. **Key derivation**: Use PBKDF2/Argon2 for passwords
4. **Padding**: Use PKCS#7, validate on decrypt

## Summary of Vulnerabilities

| Vulnerability | Severity | Exploitability |
|---------------|----------|----------------|
| Weak key generation | CRITICAL | Trivial (< 1s) |
| ECB mode | HIGH | Easy (pattern visible) |
| No authentication | MEDIUM | Enables tampering |
| Predictable structure | LOW | Aids template attacks |

### Key Takeaway
**Even strong encryption (AES-128) becomes weak with:**
- Poor key generation
- Wrong mode of operation (ECB)
- Lack of authentication

Always use established libraries and follow best practices!